In [1]:
import random
from IPython.display import display, HTML
import ipywidgets as widgets

# Ορισμός νομισμάτων και κύκλων
coin_names = ['ν1','ν2','ν3','ν4','ν5','ν6','ν7']
coins = {name: 'Κ' for name in coin_names}

circles = [
    ['ν1','ν2','ν3','ν4'],
    ['ν2','ν5','ν6','ν4'],
    ['ν3','ν6','ν7','ν4']
]

# Κανόνες
def apply_rule_a(coins, circle):
    for k in circle:
        coins[k] = 'Γ' if coins[k] == 'Κ' else 'Κ'
    return coins

def apply_rule_b(coins, circle):
    for k in circle:
        if coins[k] == 'Κ':
            coins[k] = 'Γ'
    return coins

rule_funcs = [apply_rule_a, apply_rule_b]
rule_labels = ['a', 'b']

# Widget για την εμφάνιση ενός βήματος
def get_state_widget(coins, active_circle, rule_label, step_number):
    html = f"<strong>Βήμα {step_number} — Εφαρμόστηκε κανόνας <span style='color:blue'>{rule_label}</span> στον κύκλο {circles.index(active_circle)+1}:</strong><br>"
    for k in sorted(coins.keys()):
        color = "red" if k in active_circle else "black"
        html += f"<span style='color:{color}'>{k}:{coins[k]}</span> "
    return widgets.HTML(html)

# Αρχικές καταστάσεις
initial_state = coins.copy()
states = [initial_state]
actions = []

# Widgets
latest_output = widgets.VBox()
history_output = widgets.VBox()
history_output.layout = widgets.Layout(height='300px', overflow_y='auto', border='1px solid gray', padding='5px')

# Λογική κουμπιού
step_number = 0

def on_next_click(b):
    global coins, step_number
    step_number += 1

    circle_idx = random.randint(0, 2)
    rule_idx = random.randint(0, 1)
    rule_func = rule_funcs[rule_idx]
    rule_label = rule_labels[rule_idx]
    active_circle = circles[circle_idx]

    # Εφαρμογή κανόνα
    coins = rule_func(coins.copy(), active_circle)
    states.append(coins.copy())
    actions.append((rule_label, active_circle))

    # Νέο widget για το τρέχον βήμα
    new_widget = get_state_widget(coins, active_circle, rule_label, step_number)

    # Ενημέρωση τρέχοντος βήματος (χωρίς flicker)
    latest_output.children = (new_widget,)

    # Ενημέρωση ιστορικού (αντιστρόφως)
    if step_number >1:
        prev_widget = get_state_widget(states[step_number - 1], actions[step_number - 1][1], actions[step_number - 1][0], step_number - 1)
        history_output.children = (prev_widget,) + history_output.children

# Κουμπί
next_button = widgets.Button(description="Επόμενο βήμα", button_style='info')
next_button.on_click(on_next_click)

# Εμφάνιση όλων
# Ενοποίηση όλων σε VBox για ορθή εμφάνιση στο Voilà
ui = widgets.VBox([
    widgets.HTML("<h5>Ο κανόνας a αναποδογυρίζει όλα τα νομίσματα ενός κύκλου.<br> Ο κανόνας b αναποδογυρίζει μόνο τα νομίσματα με όψη Κ <br> Ο τρέχον κύκλος που εφαρμόζεται ο κανόνας εμφανίζεται με κόκκινα γράμματα.</h5>"),
    widgets.HTML("<h3>Κατάσταση Νεώτερου Βήματος:</h3>"),
    latest_output,
    next_button,
    widgets.HTML("<h4>Ιστορικό προηγούμενων βημάτων:</h4>"),
    history_output
])

display(ui)